In [ ]:
import os
from dotenv import load_dotenv

load_dotenv()

GOOGLE_API_KEY = os.getenv("GOOGLE_API_KEY")

MODEL = "gemini-1.5-flash"
# MODEL = "llama3"

In [ ]:
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain_google_genai import GoogleGenerativeAIEmbeddings
from langchain_ollama import OllamaLLM as Ollama
from langchain_community.embeddings import OllamaEmbeddings

if MODEL.startswith("gemini"):
  model = ChatGoogleGenerativeAI(api_key=GOOGLE_API_KEY, model=MODEL)
  embeddings = GoogleGenerativeAIEmbeddings(
    model="models/embedding-001",
    google_api_key=os.getenv("GOOGLE_API_KEY")
  )
else:
  model = Ollama(model=MODEL, base_url="http://localhost:11434")
  embeddings = OllamaEmbeddings(model="nomic-embed-text",base_url="http://localhost:11434")

model.invoke("tell me a joke")

C:\Users\adiap\AppData\Local\Temp\ipykernel_18244\4276412153.py:14: LangChainDeprecationWarning: The class `OllamaEmbeddings` was deprecated in LangChain 0.3.1 and will be removed in 1.0.0. An updated version of the class exists in the :class:`~langchain-ollama package and should be used instead. To use it run `pip install -U :class:`~langchain-ollama` and import as `from :class:`~langchain_ollama import OllamaEmbeddings``.
  embeddings = OllamaEmbeddings(model="nomic-embed-text",base_url="http://localhost:11434")


In [ ]:
from langchain_core.output_parsers import StrOutputParser

parser = StrOutputParser()

chain = model | parser
chain.invoke("tell me a joke")

"Why don't scientists trust atoms? \n\nBecause they make up everything!"

In [ ]:
from langchain_community.document_loaders import PyPDFLoader

loader = PyPDFLoader("Docker.pdf")
pages = loader.load_and_split()
pages

[Document(metadata={'producer': 'Skia/PDF m133', 'creator': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/133.0.0.0 Safari/537.36', 'creationdate': '2025-02-21T05:58:26+00:00', 'title': 'Definitions and basic concepts | DevOps with Docker', 'moddate': '2025-02-21T05:58:26+00:00', 'source': 'Docker.pdf', 'total_pages': 13, 'page': 0, 'page_label': '1'}, page_content='Part 1 Definitions and basic concepts\nDefinitions and basic concepts\nWhat is DevOps?\nBefore we get started with Docker let\'s lay the groundwork for learning the right mindset.\nDefining DevOps is not a trivial task but the term itself consists of two parts, Dev and Ops. Dev\nrefers to the development of software and Ops to operations. Simple definition for DevOps\nwould be that it means the release, configuring, and monitoring of software is in the hands of the\nvery people who develop it.\nA more formal definition is offered by Jabbari et al.: "DevOps is a development methodol

In [ ]:
from langchain.prompts import PromptTemplate

template = """
Answer the question based on the context below. if you can't answer the question, reply with "I don't know".

Context: {context}

Question: {question}
"""

prompt = PromptTemplate.from_template(template)
prompt.format(context="Here is some context", question="Here is a question")

'\nAnswer the question based on the context below. if you can\'t answer the question, reply with "I don\'t know".\n\nContext: Here is some context\n\nQuestion: Here is a question\n'

In [ ]:
chain = prompt | model | parser

In [ ]:
chain.invoke(
  {
    "context": "",
    "question": ""
  }
)

"I don't know."

In [ ]:
from langchain_community.vectorstores import DocArrayInMemorySearch

vectorstore = DocArrayInMemorySearch.from_documents(pages, embedding=embeddings)

d:\Python\.venv\Lib\site-packages\pydantic\_migration.py:283: UserWarning: `pydantic.error_wrappers:ValidationError` has been moved to `pydantic:ValidationError`.
  warnings.warn(f'`{import_path}` has been moved to `{new_location}`.')


In [ ]:
retriever = vectorstore.as_retriever()
retriever.invoke("tell me about Docker")

[Document(metadata={'producer': 'Skia/PDF m133', 'creator': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/133.0.0.0 Safari/537.36', 'creationdate': '2025-02-21T05:58:26+00:00', 'title': 'Definitions and basic concepts | DevOps with Docker', 'moddate': '2025-02-21T05:58:26+00:00', 'source': 'Docker.pdf', 'total_pages': 13, 'page': 1, 'page_label': '2'}, page_content='become a DevOps specialist solely from this course, but you will get the skills to help you\nnavigate in the increasingly containerized world.\nDuring this course we will focus mainly on the packaging, releasing and configuring of the\napplications. You will not be asked to plan or create new software. We will go over Docker and a\nfew technologies that you may see in your daily life, these include e.g. Redis and Postgres. See\nStackOverflow survey on how closely they correlate these technologies.\nWhat is Docker?\n"Docker is a set of platform as a service (PaaS) products that use 

In [ ]:
from operator import itemgetter
chain = {
  "context": itemgetter("question")|retriever,
  "question" : itemgetter("question")
} | prompt | model | parser

chain.invoke({
  "question": "What is Docker?"
})

'Based on the provided text, Docker is a set of tools to deliver software in containers.  Containers are packages of software that include the application and its dependencies, and are isolated from each other and the software running outside of them.'

In [ ]:
questions = [
  "What is the purpose of the course?",
  "What is DevOps?",
  "What is a container and what are the benefits of using containers?",
  "How to run containers on our machines?",
  "What programming languages are used in the course?",
]

for question in questions:
  print(f"Question: {question}")
  print(f"Answer: {chain.invoke({"question": question})}")

Question: What is the purpose of the course?
Answer: The course focuses on the packaging, releasing, and configuring of applications using Docker and related technologies like Redis and Postgres.  It will not cover planning or creating new software.
Question: What is DevOps?
Answer: According to the text, DevOps is a development methodology aimed at bridging the gap between Development and Operations, emphasizing communication and collaboration, continuous integration, quality assurance and delivery with automated deployment utilizing a set of development practices.  A simpler definition is that the release, configuring, and monitoring of software is in the hands of the developers.
Question: What is a container and what are the benefits of using containers?
Answer: Based on the provided text, a container is a package of software that includes the application and its dependencies.  The benefits of using containers include:

* **Solving the "works on my machine" problem:** Containers ens